<a href="https://colab.research.google.com/github/Asazin007/xml-to-CSV/blob/main/steeleye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import logging
import requests
import xml.etree.ElementTree as ET

class Lambda:
    def __init__(self, url=None):
        self.url = url
        self.logger = logging.getLogger('Lambda_function')
        self.logger.setLevel(logging.INFO)

    def download_link(self):
        try:
            self.resp = requests.get(self.url)
            with open('registers.xml', 'wb') as f:
                f.write(self.resp.content)
            self.tree = ET.parse('registers.xml')
            self.root = self.tree.getroot()
            self.link = None
            for item in self.root[1].iter("doc"):
                if item.find("str[@name='file_type']").text == 'DLTINS':
                    self.link = item.find("str[@name='download_link']").text
                    break
            if not self.link:
                raise Exception("Could not find download link for file_type 'DLTINS'")
            return self.link
        except Exception as e:
            self.logger.error(f"Error in download_link: {e}")
            raise e
   
   
    def zip_extraction1(self, link=None):
        try:
            self.zip_file = requests.get(self.link)
            with open('zip_file.zip', 'wb') as f:
                f.write(self.zip_file.content)

            self.xml_file = ""
            with zipfile.ZipFile('zip_file.zip', 'r') as f:
                self.xml_file = f.namelist()[0]
                f.extractall('')

            if not self.xml_file:
                raise Exception("Could not extract xml file from zip")

            return self.xml_file
        except Exception as e:
            self.logger.error(f"Error in zip_extraction: {e}")
            raise e

    def xml_to_csv(self, xml=None):
        self.new = ET.parse(xml)
        self.test = self.new.getroot()
        self.pattern = 'FinInstrmGnlAttrbts'
        self.children = ['Id', 'FullNm', 'ClssfctnTp', 'CmmdtyDerivInd', 'NtnlCcy']
        self.tag = 'Issr'
        self.rows = []
        self.cols = [self.pattern + k for k in self.children]
        self.cols.append(self.tag)
        self.parent = 'TermntdRcrd'
    
        for i in self.test.iter():
            if self.parent in i.tag:
                self.entry = [None for x in range(len(self.cols))]
                for child in i:
                    if self.pattern in child.tag:
                            for c in child:
                                for k in range(len(self.children)):
                                    if self.children[k] in c.tag:
                                        self.entry[k] = c.text
                         
                              
                    if self.tag in child.tag:
                          self.entry[5] = child.text
                self.rows.append(self.entry)

        self.df = pd.DataFrame(self.rows, columns=self.cols)
        return self.df

In [2]:
import pandas as pd
import zipfile

if __name__ == '__main__':
    url = "https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2023-01-01T00:00:00Z%20TO%202023-01-01T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100"
    p = Lambda(url)
    zip_link = p.download_link()
    xml_file = p.zip_extraction1(zip_link)
    df = p.xml_to_csv(xml_file)
    df.to_csv('output.csv')


In [3]:
data =pd.read_csv("/content/output.csv")

In [4]:
data.head()

,Unnamed: 0,FinInstrmGnlAttrbtsId,FinInstrmGnlAttrbtsFullNm,FinInstrmGnlAttrbtsClssfctnTp,FinInstrmGnlAttrbtsCmmdtyDerivInd,FinInstrmGnlAttrbtsNtnlCcy,Issr
0,0,DE000C7A2EW5,LOL5 SI 20221230 PS AM C 345.00 0,OCASPS,False,EUR,529900UT4DG0LG5R9O07
1,1,DE000C7A2EX3,LOL5 SI 20221230 PS AM P 345.00 0,OPASPS,False,EUR,529900UT4DG0LG5R9O07
2,2,DE000C7A2EY1,LOL5 SI 20221230 PS AM C 350.00 0,OCASPS,False,EUR,529900UT4DG0LG5R9O07
3,3,DE000C7A2EZ8,LOL5 SI 20221230 PS AM P 350.00 0,OPASPS,False,EUR,529900UT4DG0LG5R9O07
4,4,DE000C7A2L07,ADS5 SI 20221230 PS AM P 110.00 0,OPASPS,False,EUR,529900UT4DG0LG5R9O07


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


NoCredentialsError: ignored